In [5]:
import tempfile
import matplotlib.pyplot as plt
import pydicom
from pydicom.data import get_testdata_file
import cv2
import os
from skimage import morphology
import math
import numpy as np
from scipy import ndimage
from tqdm import tqdm
import glob as glob
import SimpleITK as sitk
import sys
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000
from pydicom.pixel_data_handlers.util import apply_modality_lut
from imagecodecs import jpeg_encode
import imagecodecs
import random
from pydicom.uid import ExplicitVRLittleEndian
from imagecodecs import jpeg_encode
import imagecodecs
import pylibjpeg
import timeit
import copy
# # CT

# In[ ]:



def sorted_files(folder):
    ins=[]
    ser=[]
    f= os.listdir(folder)
    for name in tqdm( f):
        itkimage = sitk.ReadImage(os.path.join(folder,name))
        temp= float(itkimage.GetMetaData('0020|0013')) # instance number 
        temp1= itkimage.GetMetaData('0020|000e') #series instance UID 
        ins.append(int(temp))
        ser.append(temp1)
    series=np.unique(ser)
    files=[[x for sr,_,x in sorted(zip(ser,ins,f)) if sr==s] for s in series]
    return files

def get_scan(path):
    f= sorted_files(path)
    series=[]
    files=[]
    for i in tqdm (range (len(f))):
        img=[]
        file=[]
        for filename in f[i]:
            itkimage = sitk.ReadImage(os.path.join(path,filename))
            numpyImage = sitk.GetArrayFromImage(itkimage)
            img.append(numpyImage[0,:,:])
            file.append(filename)
        series.append(img)
        files.append(file)
    return series,files

def CT_blurred_sag(image,dicom_img,rows,columns):
    imagee = image>-800
#     #plt.imshow(imagee,'gray')
#     #plt.title("image greater than -400")
#     #plt.show()

    skull = image>1200
#     #plt.imshow(skull,'gray')
#     #plt.title("skull; image greater than 1000")
#     #plt.show()

    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     #plt.imshow(gray,'gray')
#     #plt.title("contours")
#     #plt.show()

    mid = int(image.shape[0]/2)

    gray = np.where((gray==255), 0, 255)
    gray[:,mid:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,10)
#     #plt.imshow(gray1,'gray')
#     #plt.title("contours2")
#     #plt.show()
    gray1[:,mid:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)

#     #plt.imshow(out,'gray')
#     #plt.title("out")
#     #plt.show()

    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         #print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
        #print(np.min(out),np.max(out))
        
        dum = copy.copy(image)
        dum[:,:] = 0
        input_skin = np.where(gray==1, image, dum)
        #plt.imshow(input_skin,'gray')
        #plt.show()

        dum2 = copy.copy(out)
        dum2[:,:] = 0
        output_skin = np.where(gray==1, out, dum2)
        #plt.imshow(output_skin,'gray')
        #plt.show()
        return out, input_skin, output_skin
    else: 
#         #print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
        #print(np.min(out),np.max(out))
        
#     for i in range(np.shape(out)[0]):
#         if not np.any(temp[i]):
#             out[i]=np.multiply(out[i],0)
#             out = out.astype(np.int16)
    
        dum = copy.copy(dicom_img)
        dum[:,:] = 0
        input_skin = np.where(gray==1, dicom_img, dum)
        #plt.imshow(input_skin,'gray')
        #plt.show()

        dum2 = copy.copy(out)
        dum2[:,:] = 0
        output_skin = np.where(gray==1, out, dum2)
        #plt.imshow(output_skin,'gray')
        #plt.show()
        return out, input_skin, output_skin
    
def CT_blurred_axial(image, dicom_img, rows, columns):
    #print("axial")
    imagee = image>-50
#     #plt.imshow(imagee,'gray')
#     #plt.title("image greater than -50")
#     #plt.show()
    
    skull = image>500
#     #plt.imshow(skull,'gray')
#     #plt.title("skull; image greater than 500")
#     #plt.show()
    
    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     #plt.imshow(gray,'gray')
#     #plt.title("contours")
#     #plt.show()
    gray = np.where((gray==255), 0, 255)
    gray[300:,:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,7)
#     #plt.imshow(gray1,'gray')
#     #plt.title("contours2")
#     #plt.show()
    gray1[300:,:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)
    
    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         #print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
        #print(np.min(out),np.max(out))
        dum = copy.copy(image)
        dum[:,:] = 0
        input_skin = np.where(gray==1, image, dum)
        #plt.imshow(input_skin,'gray')
        #plt.show()
        #print(input_skin)
        dum2 = copy.copy(out)
        dum2[:,:] = 0
        output_skin = np.where(gray==1, out, dum2)
        #plt.imshow(output_skin,'gray')
        #plt.show()
        return out, input_skin, output_skin
    else: 
#         #print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
        #print(np.min(out),np.max(out))
        
        dum = copy.copy(dicom_img)
        dum[:,:] = 0
        input_skin = np.where(gray==1, dicom_img, dum)
        #plt.imshow(input_skin,'gray')
        #plt.show()

        dum2 = copy.copy(out)
        dum2[:,:] = 0
        output_skin = np.where(gray==1, out, dum2)
        #plt.imshow(output_skin,'gray')
        #plt.show()
        return out, input_skin, output_skin
        
def CT_blurred_coronal(image, dicom_img, rows, columns):

    imagee = image>-400
#     #plt.imshow(imagee,'gray')
#     #plt.title("image greater than -400")
#     #plt.show()

    skull = image>1000
#     #plt.imshow(skull,'gray')
#     #plt.title("skull; image greater than 1000")
#     #plt.show()

    kernel = np.ones((30, 30), np.uint8)
    imagee = cv2.morphologyEx(imagee.astype(np.uint8), cv2.MORPH_CLOSE, kernel)
    imagee = ndimage.binary_fill_holes(imagee)
    imagee = imagee.astype(np.uint8)
    contours, hierarchy = cv2.findContours(imagee, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)    
    gray = np.zeros_like(imagee)
    gray = cv2.drawContours(gray, contours,-1, 255,-1 )
#     #plt.imshow(gray,'gray')
#     #plt.title("contours")
#     #plt.show()

    mid = int(image.shape[0]/2)

    gray = np.where((gray==255), 0, 255)
    gray[:200,:] = 0
    gray1 = np.zeros_like(imagee)
    gray1 = cv2.drawContours(gray1, contours,-1, 255,10)
#     #plt.imshow(gray1,'gray')
#     #plt.title("contours2")
#     #plt.show()
    gray1[:200,:] = 0
    gray=np.where((gray1==255), gray1, gray)
    gray=np.where((gray==255),1,0)

    if np.min(image) == np.min(dicom_img) and np.max(image) == np.max(dicom_img):
#         #print("equal")
        blurred_img = cv2.GaussianBlur(image, (101, 101), 400)
        out = np.where((gray==1), blurred_img, image)
        out = np.where(skull>0,image,out)
        out = out.astype(np.int16)
        #print(np.min(out),np.max(out))
        dum = copy.copy(image)
        dum[:,:] = 0
        input_skin = np.where(gray==1, image, dum)
        #plt.imshow(input_skin,'gray')
        #plt.show()

        dum2 = copy.copy(out)
        dum2[:,:] = 0
        output_skin = np.where(gray==1, out, dum2)
        #plt.imshow(output_skin,'gray')
        #plt.show()
        return out, input_skin, output_skin
    else: 
#         #print("not equal")
        blurred_img = cv2.GaussianBlur(dicom_img, (101, 101), 50)
        out = np.where((gray==1), blurred_img, dicom_img)
        out = np.where(skull>0,dicom_img,out)
        out = out.astype(np.int16)
        #print(np.min(out),np.max(out))
        dum = copy.copy(dicom_img)
        dum[:,:] = 0
        input_skin = np.where(gray==1, dicom_img, dum)
        #plt.imshow(input_skin,'gray')
        #plt.show()

        dum2 = copy.copy(out)
        dum2[:,:] = 0
        output_skin = np.where(gray==1, out, dum2)
        #plt.imshow(output_skin,'gray')
        #plt.show()
        return out, input_skin, output_skin
    
def predictions_ct(path, output):
    if not os.path.exists(output):
        os.makedirs(output)
    output_list= []
    classUID = []
#     start_files = timeit.default_timer()
    scan_series,filenames=get_scan(path)
#     stop_files = timeit.default_timer()
#     #print("time for files: ", stop_files - start_files)
    iskin = []
    oskin = []
#     start_ai = timeit.default_timer()
    for j in tqdm(range(len(scan_series))):
        scan, names = scan_series[j],filenames[j]
        scan=np.asarray(scan)
        #print(scan.shape[0])
        #print("no.of files: ",len(names))
        dcmData = pydicom.dcmread(os.path.join(path,names[0]))
        
#         #plt.imshow(dcmData.pixel_array,'gray')
#         #plt.show()
        
        axial  =  np.array([1., 0., 0., 0., 1., 0.])
#         sagittal = np.array([-1, 0., 0., 0., 0., -1.])
        sagittal = np.array([0, 1., 0., 0., 0., -1.])
        coronal = np.array([1., 0., 0., 0., 0., -1.])
        if (0x0020,0x0037) in dcmData:
            a = (np.around(dcmData.ImageOrientationPatient, 0))
            #print(a)
            if (a == axial).all():
                iiskin = []
                ooskin = []
#                 #print("in axial")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
#                     #print("max and min of scan sitk ",np.min(img),np.max(img))
                    dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                    dicom_img = dcmData.pixel_array
#                     #print("max and min of input dicom ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    r= dcmData.Rows
                    c=dcmData.Columns
                    out, input_skin, output_skin= CT_blurred_axial(img,dicom_img,r,c)
                    iiskin.append(input_skin)
                    ooskin.append(output_skin)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
#                     #print("max and min of output ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    
#                     #plt.imshow(dcmData.pixel_array,'gray')
#                     #plt.title("axial blurred")
#                     #plt.show()
                    
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
                iskin.append(iiskin)
                oskin.append(ooskin)
            elif (a==sagittal).all():
                iiskin = []
                ooskin = []
#                 #print("in sagittal")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) ##read file to change it's pixel data to anonymized
                    r= dcmData.Rows
                    c=dcmData.Columns
                    dicom_img = dcmData.pixel_array

                    out, input_skin, output_skin = CT_blurred_sag(img,dicom_img,r,c)
                    iiskin.append(input_skin)
                    ooskin.append(output_skin)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
                    
#                     #plt.imshow(dcmData.pixel_array,'gray')
#                     #plt.title("sagittal blurred")
#                     #plt.show()
                    
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
                iskin.append(iiskin)
                oskin.append(ooskin)
            elif (a==coronal).all():
                iiskin = []
                ooskin = []
#                 #print("in coronal")
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name))
                    dicom_img = dcmData.pixel_array
#                     #print("max and min of input dicom ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    r= dcmData.Rows
                    c=dcmData.Columns
                    out, input_skin, output_skin = CT_blurred_coronal(img,dicom_img,r,c)
                    iiskin.append(input_skin)
                    ooskin.append(output_skin)
                    out = out.astype(np.int16)
                    dcmData.file_meta.TransferSyntaxUID= '1.2.840.10008.1.2.1'
                    dcmData.PixelData = out.tobytes()
#                     #print("max and min of output ",np.min(dcmData.pixel_array),np.max(dcmData.pixel_array))
                    
#                     #plt.imshow(dcmData.pixel_array,'gray')
#                     #plt.title("coronal blurred")
#                     #plt.show()
                    
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) ##save file in anonym folder
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
                iskin.append(iiskin)
                oskin.append(ooskin)
            else:
                for i in range(scan.shape[0]):
                    name = names[i]
                    img = scan[i, :, :]
                    dcmData = pydicom.dcmread(os.path.join(path,name)) 
                    des_path = os.path.join(output, name)
                    dcmData.save_as(des_path) 
                    classUID.append(str(dcmData.SOPClassUID))
                    output_list.append(des_path)
        else:
            for i in range(scan.shape[0]):
                name = names[i]
                img = scan[i, :, :]
                dcmData = pydicom.dcmread(os.path.join(path,name)) 
                des_path = os.path.join(output, name)
                dcmData.save_as(des_path) 
                classUID.append(str(dcmData.SOPClassUID))
                output_list.append(des_path)
#     stop_ai = timeit.default_timer()
#     #print("time for AI: ", stop_ai - start_ai)
#     mimeType = "application/dicom"
#     recommendation_string = {"finding": "finding","conclusion":"conclusion","recommendation":"recommendation"} 
#     return output_list, classUID, mimeType, recommendation_string
    return iskin, oskin

In [6]:
def same_range(imggr, imggf):
    min1 = np.min(imggr)
    max1 = np.max(imggr)

    # Find min and max of image2
    min2 = np.min(imggf)
    max2 = np.max(imggf)

    # Find overall min and max
    min_overall = min(min1, min2)
    max_overall = max(max1, max2)

    # Scale image1 to the overall range
    imggdd = (imggr - min1) / (max1 - min1) * ((max_overall - min_overall) + min_overall)

    # Scale image2 to the overall range
    imggff = (imggf - min2) / (max2 - min2) * ((max_overall - min_overall) + min_overall)
    
#     maxi = np.max(imggdd)
#     imggdd = imggdd/1024
#     imggff = imggff/1024
    return imggdd, imggff

In [7]:
ll = []
path = ["/home/minha/FARZEEN/face ann/face ct/7124319_GiSungyoon/1.2.392.200036.9116.2.6.1.48.1211376315.1667261776.240966/",
       "/home/minha/FARZEEN/face ann/face ct/7122308_HongSungok/1.2.392.200036.9116.2.6.1.48.1211376315.1660290675.708521/",
       "/home/minha/FARZEEN/face ann/face ct/7106608_KimJiyoon/1.2.392.200036.9116.2.6.1.48.1211376315.1600048726.383271/",
       "/home/minha/FARZEEN/face ann/face ct/7019115_KimHwaKyoung/1.2.392.200036.9116.2.6.1.48.1211376315.1375921545.238520/"]
for p in path:
    s = os.listdir(p)
    for ss in s:
        h = os.path.join(p,ss)
        ll.append(h)

In [11]:
from math import log10, sqrt
import cv2
import numpy as np
  
def PSNR(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    if(mse == 0):  # MSE is zero means no noise is present in the signal .
                  # Therefore PSNR have no importance.
        return 100
    max_pixel = np.max(original)#65536#np.max(np.sum(original)**2)
    psnr = 20 * log10(abs((max_pixel-1) / sqrt(mse)))
    return psnr

In [13]:
from skimage.metrics import structural_similarity
opath = "/home/minha/FARZEEN/face ann/face ct/7019115/improved/test/"
for l in ll:
    try:
        print(l)
        iskin, oskin = predictions_ct(l, opath)
        for i in range(len(iskin)):
            print(i)
            skin_mask,After_annonymization_skin_mask = same_range(iskin[i], oskin[i])
            score,diff = structural_similarity(np.array(skin_mask), np.array(After_annonymization_skin_mask), full=True)
            print (score)
#             value = PSNR(np.array(skin_mask ), np.array(After_annonymization_skin_mask ))
#             print(f"PSNR value for skin matched is {value} dB")
#             plt.imshow(iskin[i][int(len(iskin[i])/2)],'gray')
#             plt.show()
#             Power_of_signall = (np.sum(iskin)**2)
#             Power_of_noisee = (np.sum(oskin)**2)
#             SNRR = Power_of_signall/Power_of_noisee
#             print("Before Matching: ", SNRR)
#             skin_mask,After_annonymization_skin_mask = same_range(iskin, oskin)
#             Power_of_signal = (np.sum(skin_mask)**2)
#             Power_of_noise = (np.sum(After_annonymization_skin_mask)**2)
#             SNR = Power_of_signal/Power_of_noise
#             print("After Matching: ",SNR)
#             snrr=[]
#             for j in range(len(iskin[i])):
#                 skin_maskk,After_annonymization_skin_maskk = same_range(iskin[i][j], oskin[i][j])
#                 Power_of_signalll = (np.sum(skin_maskk)**2)
#                 Power_of_noiseee = (np.sum(After_annonymization_skin_maskk)**2)
#                 SNRRr = Power_of_signalll/Power_of_noiseee
#         #         print(SNRR)
#                 snrr.append(SNRRr)
#             newList = list()
#             for element in snrr:
#                 if str(element) != "nan":
#                     newList.append(element)
#             ss = sum(newList) / len(newList)
#             print("avg per slice matched",ss)
#             print("###########################")                
            print("***************                                                   *****************")
    except Exception as e:
        print("Error in: ",l)
        print(e)

/home/minha/FARZEEN/face ann/face ct/7124319_GiSungyoon/1.2.392.200036.9116.2.6.1.48.1211376315.1667261776.240966/Neck_2_0_post_35secCONTRASTCoronal_3_CE


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.84s/it]


0
0.6904180842951166
***************                                                   *****************
/home/minha/FARZEEN/face ann/face ct/7124319_GiSungyoon/1.2.392.200036.9116.2.6.1.48.1211376315.1667261776.240966/3_post_1m20sCONTRAST_CE


100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 131.14it/s]


/home/minha/FARZEEN/face ann/face ct/7124319_GiSungyoon/1.2.392.200036.9116.2.6.1.48.1211376315.1667261776.240966/Neck_2_0_post_35secCONTRAST_CE


...

/home/minha/FARZEEN/face ann/face ct/7019115_KimHwaKyoung/1.2.392.200036.9116.2.6.1.48.1211376315.1375921545.238520/Head_3_0_Sagittal_1


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.91s/it]


0
-0.008781479423725156
***************                                                   *****************
/home/minha/FARZEEN/face ann/face ct/7019115_KimHwaKyoung/1.2.392.200036.9116.2.6.1.48.1211376315.1375921545.238520/Head_3_0_Coronal_5


100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.37s/it]


0
-2.2422236884326408e-08
***************                                                   *****************
